In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [5]:
df_costs = pd.read_csv('https://raw.githubusercontent.com/ZorroHZR/MMAI-5000/main/Omis%20A3/costs.csv')


# Extract data from DataFrame
fixed_costs = df_costs['Fixed'].tolist()
variable_costs = df_costs['Variable'].tolist()

# Create a new model
model = gp.Model("HealthLink_Distribution")

# Create decision variables
units = model.addVars(range(27), vtype=GRB.INTEGER, name="units")
sites = model.addVars(range(27), vtype=GRB.BINARY, name="sites")

# Set objective function
fixed_cost_term = gp.quicksum(fixed_costs[i] * sites[i] for i in range(27))
variable_cost_term = gp.quicksum(variable_costs[i] * units[i] for i in range(27))
model.setObjective(fixed_cost_term + variable_cost_term, GRB.MINIMIZE)

# Add constraints
# Constraint: Total units distributed must match the total inventory
model.addConstr(gp.quicksum(units[i] for i in range(27)) == 2900000, name="inventory_match")

# Constraint: Each open site must receive at least 175,000 units but no more than 375,000 units
for i in range(27):
    model.addConstr(175000 * sites[i] <= units[i], name=f"min_units_site_{i}")
    model.addConstr(units[i] <= 375000 * sites[i], name=f"max_units_site_{i}")
# Constraint: At least four sites must be chosen from sites 6 to 16
model.addConstr(gp.quicksum(sites[i] for i in range(5, 16)) >= 4, name="min_sites_6_16")
# Constraint: No more than 6 sites can be chosen from even-numbered sites
model.addConstr(gp.quicksum(sites[i] for i in range(0, 27, 2)) <= 6, name="max_even_sites")
# Constraint: If site 1 or 2 is open, then sites 5, 6, and 7 must be closed
model.addConstr(sites[0] + sites[1] + sites[4] + sites[5] + sites[6] <= 1, name="sites_1_2_5_6_7")
# Constraint: If any site from 19 to 22 is open, then sites 24, 26, and 27 must be closed
model.addConstr(gp.quicksum(sites[i] for i in range(18, 22)) + sites[23] + sites[25] + sites[26] <= 4, name="sites_19_22_24_26_27")
# Constraint: If any site from 1 to 5 is open, then at least one odd-numbered site from 21 to 27 must be open
model.addConstr(gp.quicksum(sites[i] for i in range(5)) <= gp.quicksum(sites[i] for i in range(20, 27, 2)), name="sites_1_5_odd_21_27")
# Constraint: The number of open sites from 1 to 14 must equal the number of open sites from 15 to 27
model.addConstr(gp.quicksum(sites[i] for i in range(14)) == gp.quicksum(sites[i] for i in range(14, 27)), name="equal_open_sites")
# Constraint: The total units allocated to sites 1-9 and 19-27 must be equal
model.addConstr(gp.quicksum(units[i] for i in range(9)) == gp.quicksum(units[i] for i in range(18, 27)), name="equal_allocated_units")
# Optimize the model
model.optimize()


# Calculate fixed costs and variable costs
total_fixed_costs = sum(fixed_costs[i] * sites[i].x for i in range(27))
total_variable_costs = sum(variable_costs[i] * units[i].x for i in range(27))
# Print the results
print(f"Optimal cost: ${model.objVal:.2f}")
print(f"Fixed costs: ${total_fixed_costs:.2f}")
print(f"Variable costs: ${total_variable_costs:.2f}")
print("Units:")
for i in range(27):
    if sites[i].x > 0.5:
        print(f"Site {i+1}: {units[i].x} units")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 62 rows, 54 columns and 226 nonzeros
Model fingerprint: 0x65ef4fc6
Variable types: 0 continuous, 54 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+05]
  Objective range  [2e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+06]
Found heuristic solution: objective 1.676125e+07
Presolve time: 0.00s
Presolved: 62 rows, 54 columns, 226 nonzeros
Variable types: 0 continuous, 54 integer (27 binary)
Found heuristic solution: objective 1.676125e+07

Root relaxation: objective 1.216850e+07, 39 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T